In [50]:
import os
from sklearn import preprocessing
import pandas as pd
import numpy as np
import torch
from botorch.utils.multi_objective import pareto
import numpy as np
import matplotlib.pyplot as plt

### 1. Setting up the MRL scaler  
The Optimus5Prime provided in the directory is trained using a normalized MRL values, and therefore predict normalized values. The scaler loaded here will transform the noramlized values back to the actual MRL values.

In [6]:
label_data = pd.read_csv(os.path.join("./data/GSM3130435_egfp_unmod_1_processed.tsv"), sep="\t")["rl"]
scaler = preprocessing.StandardScaler().fit(np.array(label_data).reshape(-1,1))

In [48]:
def convert_obj_to_real(df_tmp, column_name, obj_name):
    if obj_name == "AGC content [%]":
        obj_val = np.array(df_tmp[column_name])
        seq_length = len(df_tmp["cand_seq"].iloc[0])
        obj_val = (seq_length - obj_val) / seq_length * 100
    elif obj_name == "MRL":
        obj_val = np.array(df_tmp[column_name])
        obj_val = scaler.inverse_transform(-obj_val.reshape(-1,1)).flatten()
    elif obj_name == "G4 score":
        obj_val = - np.array(df_tmp[column_name])
    elif obj_name == "in vitro stability":
        obj_val = - np.array(df_tmp[column_name])
    return obj_val

def format_to_rna(tmp_array):
    for i, seq in enumerate(tmp_array):
        tmp_array[i] = seq.replace("T", "U")
    return tmp_array
    
def format_init(df_init):
    df_init = df_init.rename(
        columns={
            "sequence":"cand_seq",
            "nonU_cotent":"obj_val_0",
            "mrl":"obj_val_1",
            "G4":"obj_val_2",
            "degradation":"obj_val_3"
        }
    )
    df_init["round_idx"] = [0,]*len(df_init)
    df_init["cand_uuid"] = ["",]*len(df_init)
    df_init["cand_ancestor"] = ["",]*len(df_init)
    # df_init = df_init.reindex(columns=["round_idx","cand_uuid","cand_ancestor","cand_seq","obj_val_0","obj_val_1","obj_val_2","obj_val_3"])
    return df_init

# assume all objectives are to be maximized
def pareto_ranking(df_all, df_pareto, obj_names):
    rank_scores = []
    for i in range(len(df_pareto)):
        df_tmp = df_all.copy()
        for tmp_obj in obj_names:
            df_tmp = df_tmp[df_tmp[tmp_obj]<=df_pareto[tmp_obj].iloc[i]]
        rank_scores.append(len(df_tmp)+1)
    df_pareto["score_pareto_ranking"] = rank_scores
    df_pareto = df_pareto.sort_values(by="score_pareto_ranking", ascending=False)
    return df_pareto

# assume all objectives are to be maximized
def pareto_R_method(df_pareto, obj_names, obj_rank_values):
    # calculate rank weights for objectives
    obj_rank = pd.DataFrame(obj_rank_values, columns=["rank"])
    obj_rank["rank_inv"] = 1/obj_rank["rank"]

    tmp_weights = []
    obj_rank = obj_rank.sort_values(by="rank")
    tmp_values = np.sort(np.unique(obj_rank["rank"]))
    for tmp_value in tmp_values:
        tmp_rows = obj_rank[obj_rank["rank"]==tmp_value]
        tmp_weight = 1/np.sum(obj_rank[obj_rank["rank"]<=tmp_value]["rank_inv"])
        tmp_weights.extend([tmp_weight]*len(tmp_rows))

    obj_rank["tmp_weight"] = tmp_weights
    obj_rank["rank_weight"] = obj_rank["tmp_weight"] / np.sum(obj_rank["tmp_weight"])
    obj_rank_weights = obj_rank["rank_weight"].values

    # calculate rank weights for pareto points
    df_tmp = df_pareto.copy()
    for tmp_obj in obj_names:
        df_tmp = df_tmp.sort_values(by=tmp_obj, ascending=False)
        df_tmp["rank_{}".format(tmp_obj)] = np.arange(1,len(df_tmp)+1)
        tmp_values = np.sort(np.unique(df_tmp[tmp_obj]))[::-1]
        if len(tmp_values)==len(df_tmp):
            pass
        else:
            new_rank = []
            for tmp_value in tmp_values:
                tmp_rows = df_tmp[df_tmp[tmp_obj]==tmp_value]
                if len(tmp_rows)>1:
                    new_rank_value = np.sum(tmp_rows["rank_{}".format(tmp_obj)].values)/len(tmp_rows)
                    new_rank.extend([new_rank_value]*len(tmp_rows))
                else:
                    new_rank.append(tmp_rows["rank_{}".format(tmp_obj)].values[0])
            df_tmp["rank_{}".format(tmp_obj)] = new_rank
        df_tmp["rank_inv_{}".format(tmp_obj)] = 1/df_tmp["rank_{}".format(tmp_obj)]

        tmp_weights = []
        tmp_values = np.sort(np.unique(df_tmp["rank_{}".format(tmp_obj)]))
        for tmp_value in tmp_values:
            tmp_rows = df_tmp[df_tmp["rank_{}".format(tmp_obj)]==tmp_value]
            tmp_weight = 1/np.sum(df_tmp[df_tmp["rank_{}".format(tmp_obj)]<=tmp_value]["rank_inv_{}".format(tmp_obj)])
            tmp_weights.extend([tmp_weight]*len(tmp_rows))
        df_tmp["tmp_weight"] = tmp_weights
        df_pareto["rank_weight_{}".format(tmp_obj)] = df_tmp["tmp_weight"] / np.sum(df_tmp["tmp_weight"])

    df_pareto["score_Rmethod"] = np.zeros(len(df_pareto))
    for i, tmp_obj in enumerate(obj_names):
        df_pareto["score_Rmethod"] += df_pareto["rank_weight_{}".format(tmp_obj)] * obj_rank_weights[i]
    df_pareto = df_pareto.sort_values(by="score_Rmethod", ascending=False)
    return df_pareto

from itertools import product
def generate_MIPS_weights(N, w):
    max_multiples = int(1 / w)
    possible_values = [i * w for i in range(max_multiples + 1)]
    all_combinations = product(possible_values, repeat=N)
    
    # Filter combinations where the sum of elements is exactly 1
    valid_combinations = [comb for comb in all_combinations if np.isclose(sum(comb), 1)]
    result_array = np.array(valid_combinations)
    return result_array

# assume all objectives are to be maximized
def pareto_MIPS(df_pareto, obj_names, weights=[]):
    # parameter in augmented weighted Tchebycheff
    rho = 0.0
    # window of weights
    wind = 0.01
    # value of Tstar
    Tstar = 0.01

    obj_values = - np.array(df_pareto[obj_names])
    # min-max normalization
    obj_norm = (obj_values - np.min(obj_values, axis=0))/(np.max(obj_values, axis=0) - np.min(obj_values, axis=0))
    num_samples = obj_norm.shape[0]

    if len(weights)==0:
        weights = generate_weights(len(obj_names), w=wind)

    FT_list=[]
    pareto_list = []
    # H_all = []
    for weight in weights:
        H = np.max(np.multiply(np.tile(np.array(weight), obj_norm.shape[0]).reshape(-1,len(weight)), obj_norm), axis=1)
        H += rho * np.sum(obj_norm, axis=1)
        
        Emin, Emax = np.min(H), np.max(H)
        pareto_list.append(np.argmin(H))
        # H_all.append((H-Emin)/(Emax-Emin))
        
        H_norm = (H-Emin)/(Emax-Emin)
        Z = 0.0
        for i in range(num_samples):
            Z += np.exp(-H_norm[i]/Tstar)

        if Z==0.0:
            FT_list.append(0.0)
        else:
            FT_list.append(- Tstar*(np.log(Z)))

    arg_index = np.array(FT_list).argsort()[::-1]
    ranking_index = []
    ranking_MIPS = []

    for i in range(len(arg_index)):
        if pareto_list[arg_index[i]] not in ranking_index:
            ranking_index.append(pareto_list[arg_index[i]])
            ranking_MIPS.append(FT_list[arg_index[i]])
    
    MIPS_selected, MIPS_scores = np.zeros(len(df_pareto)), np.full(len(df_pareto), fill_value=-10000.0)
    MIPS_selected[ranking_index] += 1
    MIPS_scores[ranking_index] = ranking_MIPS
    df_pareto["MIPS_selected"] = MIPS_selected
    df_pareto["MIPS_score"] = MIPS_scores
    return df_pareto

### 2. Retrieving generated sequences  
Retrieving generated sequences during LaMBO trainings.  
For each corresponding run, the table of generated sequences should be uploaded to your wandb workspace as "task_mugd_cnn/candidates" or you can find them under your local wandb directory such as `./wandb/RUNTITLE/files/media/table/task_mugd_cnn/candidates_ID.table.json`. 

In [51]:
dict_df = {
    "init":None,
    "CNN":None,
    "BERT":None,
    "DNABERT":None,
}

initset_flg = "initset3"
seed_flg = 1
data_dir = "./generated_seqs"
filelist = []
filelist.append(
    f"../cache_dir/Sample2019_unmod1_{initset_flg}_numsample512_ConstructSample2019EGFP.csv"
)
for tmp_file in os.listdir(data_dir):
    if initset_flg in tmp_file and f"seed{seed_flg}" in tmp_file:
        filelist.append(os.path.join(data_dir, tmp_file))
obj_columns = ["obj_val_0","obj_val_1","obj_val_2","obj_val_3"]
obj_names = ["AGC content [%]", "MRL", "G4 score", "in vitro stability"]

for tmp_file in filelist:
    df_tmp = pd.read_csv(tmp_file)
    df_save = pd.DataFrame([], columns=obj_names)
    new_points = None
    if "Sample2019" in tmp_file:
        df_tmp = format_init(df_tmp)
    for obj_column, obj_name in zip(obj_columns, obj_names):
        tmp_obj = convert_obj_to_real(df_tmp, obj_column, obj_name)
        if new_points is None:
            new_points = np.expand_dims(tmp_obj, -1)
        else:
            new_points = np.concatenate([new_points, np.expand_dims(tmp_obj, -1)], -1)
    df_save["sequence"] = format_to_rna(np.array(df_tmp["cand_seq"]))
    df_save[obj_names] = new_points
    if not "Sample2019" in tmp_file:
        df_save["round_idx"] = df_tmp["round_idx"]
    else:
        df_save["round_idx"] = [-1,]*len(df_tmp)
    
    if "CNN" in tmp_file:
        dict_df["CNN"] = df_save
    elif "DNABERT" in tmp_file:
        dict_df["DNABERT"] = df_save
    elif "BERT" in tmp_file:
        dict_df["BERT"] = df_save
    else:
        dict_df["init"] = df_save

### 3. Ranking generated sequences
Each cells below will rank generated sequences based on the corresponding method. The methods are either Pareto frontier, Pareto ranking, R-method, or MIPS scores.  

In [52]:
dict_pareto = {
    "CNN":None,
    "BERT":None,
    "DNABERT":None,
    "init":None,
}

for key, df in dict_df.items():
    print(f"Processing {key} ...")
    if not key=="init":
        df = pd.concat([df, dict_df["init"]], axis=0)
        all_points = df[obj_names].to_numpy()
    else:
        all_points = df[obj_names].to_numpy()
    pareto_mask = pareto.is_non_dominated(torch.tensor(all_points), )
    pareto_idx = np.argsort(all_points[pareto_mask][:,0])
    df_tmp = df[pareto_mask.numpy()].iloc[pareto_idx,:]
    dict_pareto[key] = df_tmp

Processing init ...


Processing CNN ...
Processing BERT ...
Processing DNABERT ...


In [45]:
dict_pareto_ranking = {
    "CNN":None,
    "BERT":None,
    "DNABERT":None,
    "init":None,
}
for key, df in dict_df.items():
    print(f"Processing {key} ...")
    if not key=="init":
        df = pd.concat([dict_df["init"], df], axis=0)
    df_pareto_ranking = pareto_ranking(df, df, obj_names)
    dict_pareto_ranking[key] = df_pareto_ranking

Processing init ...
Processing CNN ...
Processing BERT ...
Processing DNABERT ...


In [46]:
obj_rank_values = [2, 1, 4, 3]
dict_R_method = {
    "CNN":None,
    "BERT":None,
    "DNABERT":None,
    "init":None,
}
for key, df in dict_df.items():
    print(f"Processing {key} ...")
    if not key=="init":
        df = pd.concat([dict_df["init"]], df, axis=0).reset_index(drop=True)
    df_R_method = pareto_R_method(df, obj_names, obj_rank_values)
    dict_R_method[key] = df_R_method

Processing init ...
Processing CNN ...
Processing BERT ...
Processing DNABERT ...


In [49]:
# pre-computing weights for calculating MIPS scores. This would take ~40min for N=4 and w=0.01.
# weights_4 = generate_MIPS_weights(N=4, w=0.01)
# np.save(weights_4, "./data/MIPS_weights_4.npy")

# This step will take ~20min.
weights_4 = np.load("./data/MIPS_weights_4.npy")
dict_MIPS = {
    "CNN":None,
    "BERT":None,
    "DNABERT":None,
    "init":None,
}
for key, df in dict_df.items():
    print(f"Processing {key} ...")
    if not key=="init":
        df = pd.concat([dict_df["init"]], df, axis=0).reset_index(drop=True)
    df_MIPS = pareto_MIPS(df, obj_names, weights=weights_4)
    dict_MIPS[key] = df_MIPS

Processing init ...
Processing CNN ...
Processing BERT ...
Processing DNABERT ...


### 4. Generate figures

In [56]:
dict_colors = {"CNN":np.array([79, 173, 234])/255,
             "BERT":np.array([239, 206, 69])/255,
             "DNABERT":np.array([127, 23, 14])/255}
pareto_ranking_top_N = 10
SAVEFIG=True

for tmp_model in ["CNN", "BERT", "DNABERT"]:
    for pareto_or_ranking in ["pareto_front", "pareto_ranking", "pareto_R_method", "pareto_MIPS"]:
        print(f"Processing {tmp_model} {pareto_or_ranking}")
        tmp_color = dict_colors[tmp_model]

        fig = plt.figure(figsize=(16,8),
                            facecolor=[0,0,0,0])
        ax = fig.subplots(nrows=1, ncols=1)

        obj_names = ["AGC content [%]", "MRL", "G4 score", "in vitro stability"]
        plt.rcParams["axes.grid"] = False
        ax.set_facecolor([1,1,1])
        ax.spines["right"].set_visible(False)
        ax.spines["left"].set_visible(False)
        ax.spines["top"].set_visible(False)
        ax.spines["bottom"].set_visible(False)
        ax.yaxis.set_visible(False)

        x_offset = 0.05
        obj_labels = ["%AGC", "MRL", "G4", "Stablity"]
        ax.set_xlim([0, 1])
        x_vis = np.linspace(x_offset, 1-x_offset, len(obj_labels))
        ax.set_xticks(x_vis)
        ax.set_xticklabels(obj_labels, fontsize=18)
        ax.tick_params(axis="x", which="major", pad=10)

        y_mins = np.array([60, 0.0, -1.0, -0.44])
        y_maxs = np.array([100, 9.0, 0, -0.41])
        x_distance = (1 - 2*x_offset)/(len(obj_names)-1)
        y_ranges = y_maxs - y_mins
        y_pad = 0.02
        y_mins_padded = y_mins - y_ranges * y_pad
        y_maxs_padded = y_maxs + y_ranges * y_pad
        for i, obj_name in enumerate(obj_names):
            ymin = y_mins_padded[i]
            ymax = y_maxs_padded[i]
            new_twin = ax.twinx()
            new_twin.spines.right.set_position(("axes", x_offset + x_distance*i))
            new_twin.set_ylim([ymin, ymax])
            new_twin.spines["right"].set_color("black")
            new_twin.tick_params(direction="inout", length=10, labelsize=14)
            new_twin.tick_params(axis="y", pad=5)
            new_twin.spines["left"].set_visible(False)
            if i==0:
                y_ticks = np.arange(y_mins[i], y_maxs[i]+1, 10)
                new_twin.set_yticks(y_ticks, labels=[str(y_mins[i])] + [""]*(len(y_ticks)-2) + [str(y_maxs[i])])
            elif i==1:
                y_ticks = np.arange(y_mins[i], y_maxs[i]+0.1, 1.0)
                new_twin.set_yticks(y_ticks, labels=[str(y_mins[i])] + [""]*(len(y_ticks)-2) + [str(y_maxs[i])])
            elif i==2:
                y_ticks = np.arange(y_mins[i], y_maxs[i]+0.01, 0.1)
                new_twin.set_yticks(y_ticks, labels=["$\minus$1.0",] + [""]*(len(y_ticks)-2) + ["0.0"])
            elif i==3:
                y_ticks = np.arange(y_mins[i], y_maxs[i]+0.001, 0.01)
                new_twin.set_yticks(y_ticks, labels=["$\minus$0.44",] + [""]*(len(y_ticks)-2) + ["$\minus$0.41"])

        y_ranges_padded = y_maxs_padded - y_mins_padded
        df_vis = dict_df["init"].drop_duplicates("sequence")
        for i in range(len(df_vis)):
            y_vis = df_vis[obj_names].iloc[i]
            y_vis = (y_vis - y_mins) / y_ranges
            ax.plot(x_vis, y_vis, linestyle="-", color="gray", alpha=0.1)

        ax.set_ylim([-y_pad, 1+y_pad])

        if pareto_or_ranking == "pareto_front":
            df_vis = dict_pareto[tmp_model].drop_duplicates("sequence")
        elif pareto_or_ranking == "pareto_ranking":
            df_vis = dict_pareto_ranking[tmp_model].drop_duplicates("sequence").iloc[:pareto_ranking_top_N]
        elif pareto_or_ranking == "pareto_R_method":
            df_vis = dict_R_method[tmp_model].drop_duplicates("sequence").iloc[:pareto_ranking_top_N]
        elif pareto_or_ranking == "pareto_MIPS":
            df_vis = dict_MIPS[tmp_model].drop_duplicates("sequence")
            df_vis = df_vis.sort_values(by="MIPS_score", ascending=False).iloc[:pareto_ranking_top_N]

        print(f"\tNumber of top-ranked sequences: {len(df_vis)}")
        for i in range(len(df_vis)):
            y_vis = df_vis[obj_names].iloc[i]
            y_vis = (y_vis - y_mins) / y_ranges
            ax.plot(x_vis, y_vis, linestyle="-", color=tmp_color, alpha=0.6, linewidth=2)

        if pareto_or_ranking == "pareto_front":
            ax.set_title(f"{tmp_model} Pareto Front", fontsize=20, pad=10)
        elif pareto_or_ranking == "pareto_ranking":
            ax.set_title(f"{tmp_model} Pareto Ranking Top{pareto_ranking_top_N}", fontsize=20, pad=10)
        elif pareto_or_ranking == "pareto_R_method":
            ax.set_title(f"{tmp_model} R Method Top{pareto_ranking_top_N}", fontsize=20, pad=10)
        elif pareto_or_ranking == "pareto_MIPS":
            ax.set_title(f"{tmp_model} MIPS Top{pareto_ranking_top_N}", fontsize=20, pad=10)
        fig.tight_layout()
        if SAVEFIG:
            fig.savefig(f"./figs/{pareto_or_ranking}_ppd_{tmp_model}.png", dpi=300)
        plt.close()

Processing CNN pareto_front
	Number of top-ranked sequences: 38
Processing CNN pareto_ranking
	Number of top-ranked sequences: 10
Processing CNN pareto_R_method
	Number of top-ranked sequences: 10
Processing CNN pareto_MIPS
	Number of top-ranked sequences: 10
Processing BERT pareto_front
	Number of top-ranked sequences: 95
Processing BERT pareto_ranking
	Number of top-ranked sequences: 10
Processing BERT pareto_R_method
	Number of top-ranked sequences: 10
Processing BERT pareto_MIPS
	Number of top-ranked sequences: 10
Processing DNABERT pareto_front
	Number of top-ranked sequences: 49
Processing DNABERT pareto_ranking
	Number of top-ranked sequences: 10
Processing DNABERT pareto_R_method
	Number of top-ranked sequences: 10
Processing DNABERT pareto_MIPS
	Number of top-ranked sequences: 10
